#Исходный код RAG'a

In [1]:
#Сам добовлял устанавливаемые библиотеки для рассчета метрик
!pip install gigachain -q
!pip install sentence-transformers -q
!pip install faiss-cpu -q
!pip install gigachain_community -q
!pip install gigachain-core -q
!pip install pypdf -q
!pip install nltk -q
!pip install rouge-score -q
!pip install transformers datasets -q
!pip install torchmetrics -q

In [2]:
from getpass import getpass
from langchain.chat_models import GigaChat

In [3]:
gigachat_api_key = getpass(prompt='Введите API ключ от GigaChat')

Введите API ключ от GigaChat··········


In [4]:
# Инициализируем языковую модель GigaChat
# verify_ssl_certs=False – без использования сертификатов Минцифры
llm = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01)

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

In [6]:
# Список путей к документам с ответами на вопросы
document_paths_new = [
    "8).pdf",
    "18).pdf",
    "24).pdf",
    "25).pdf",
    "28).pdf"
]

In [7]:
# Загрузка всех документов
from langchain_community.document_loaders import PyPDFLoader
documents = []
for path in document_paths_new:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

In [8]:
# Разделение текста на части
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_docs = text_splitter.split_documents(documents)
print(f"Total documents: {len(split_docs)}")

Total documents: 219


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [10]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru"
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_documents(split_docs, hf_embeddings_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

##Создание списков вопросов и ответов данных и сгенерированных LLM

In [11]:
from langchain.chains import RetrievalQA

In [12]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [16]:
message = []
for e in questions:
    message.append(qa_chain({"query": e})['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [13]:
import pandas as pd

In [14]:
questions_df = pd.read_excel("q_data.xlsx", nrows=3).Question
answers_df = pd.read_excel("q_data.xlsx", nrows=3).Answer

In [15]:
answers = []
for e in answers_df:
  answers.append(e)

questions = []
for e in questions_df:
  questions.append(e)

#BLEUMetric

Вторая попытка реализации (Удачная)

In [ ]:
#Попытался написать функцию, что то не работает, скорее всего из-за ссылок, но тратить время не решение проблемы не стал
#def qwer(text):
#  res = []
#  for e in range(len(text)):
#    res.append(text[e].split)
#  return res

In [17]:
#Импортируем функцию вычисления BLEU
from nltk.translate.bleu_score import corpus_bleu

#Подготовка данных в необходимую форму для вычесления метрики BLEU (приведение к токенизированному виду)
translations = []
for e in range(len(answers)):
  translations.append(answers[e].split())

#Оборачиваем ответы в необходимую форму для вычесления метрики BLEU
references = [[[ref]] for ref in message]

#Костыльное решение, такое, чтобы привелось в необходумую форму
references_list = []
i = 0
for e in range(len(references)):
  references_list.append([])
  references_list[i].append(references[e][0][0].split())
  i = i + 1

#Вычисляем BLEU Score
bleu_score_corpus = corpus_bleu(references_list, translations)
print("Corpus BLEU Score: ", bleu_score_corpus)

Corpus BLEU Score:  3.151166241307109e-79


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [18]:
#Откуда взят код
references = [['I', 'love', 'eating', 'ice', 'cream'], ['He', 'enjoys', 'eating', 'cake']]
translations = [['I', 'love', 'eating', 'ice', 'cream'], ['He', 'likes', 'to', 'eat', 'cake']]

# Create a list of reference lists
references_list = [[ref] for ref in references]

#Вычисляем BLEU Score
bleu_score_corpus = corpus_bleu(references_list, translations)
print("Corpus BLEU Score: ", bleu_score_corpus)

Corpus BLEU Score:  0.5438786529686386


#ROUGE

In [19]:
from rouge_score import rouge_scorer

In [20]:
r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rougeL, другой компонент оценки ROUGE, вычисляет самую длинную общую подпоследовательность (LCS) между системными и справочными сводками. В отличие от N-грамм, LCS измеряет максимальную последовательность слов (не обязательно смежных), которые появляются в обоих сводках. Он предлагает более гибкую меру сходства и помогает фиксировать общую информацию, выходящую за рамки строгих дословных совпадений.

Не разобрался, какой из rouge1, rouge2 является N, а какой S

ROUGE-N - это компонент оценки ROUGE, который количественно определяет перекрытие N-граммов, непрерывных последовательностей из N элементов (обычно слов или символов), между сгенерированной системой сводкой и справочной сводкой. Он дает представление о точности и повторяемости выходных данных системы, учитывая совпадающие N-граммовые последовательности.

ROUGE-S фокусируется на пропуск-биграммах. Пропуск-биграмма - это пара слов в предложении, которая допускает пробелы или слова между ними. Этот компонент определяет перекрытие пропусков и биграмм между системным и справочным резюме, позволяя оценить сходство структуры на уровне предложений. Он может фиксировать перефразирующие связи между предложениями и давать представление о способности системы передавать информацию с гибким порядком слов.

In [78]:
#Код, на который опирался
candidate_summary = "the cat was found under the bed"
reference_summary = "the cat was under the bed"
r_scores = r_scorer.score(reference_summary, candidate_summary)
for key in r_scores:
    print(f'{key}: {r_scores[key]}')
r_scores['rouge1']

rouge1: Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)
rouge2: Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)
rougeL: Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)


Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)

In [21]:
df = pd.DataFrame(columns=['Question', 'Answer', 'LLM_Answer', 'rouge1', 'rouge2', 'rougeL'])
for e in range(len(questions)):
  candidate_summary = message[e]
  reference_summary = answers[e]
  r_scores = r_scorer.score(reference_summary, candidate_summary)
  new_row = {'Question': questions[e], 'Answer': answers[e], 'LLM_Answer': message[e], 'rouge1': r_scores['rouge1'], 'rouge2': r_scores['rouge2'], 'rougeL': r_scores['rougeL']}
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [22]:
df

,Question,Answer,LLM_Answer,rouge1,rouge2,rougeL
0,СБ ЗОКИИ: какие ограничения существуют по допу...,прямые ограничения по допуску иностранных граж...,В значимом объекте не допускается наличие удал...,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
1,Серия вебинаров: моделирование угроз должно вы...,первичное моделирование угроз безопасности инф...,Моделирование угроз должно выполняться до разр...,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
2,СБ ЗОКИИ: допускается ли использовать функции ...,"да, допускается, однако в отношении используем...",Не знаю,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"


#Семантическая похожесть на основе Sentence Transformers

##Опора

In [23]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Example sentences (already preprocessed)
tokens1 = ["[CLS]", "i", "like", "coding", "in", "python", ".", "[SEP]"]
tokens2 = ["[CLS]", "python", "is", "my", "favorite", "programming", "language", ".", "[SEP]"]


In [ ]:
# Convert tokens to input IDs
input_ids1 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens1)).unsqueeze(0)  # Batch size 1
input_ids2 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens2)).unsqueeze(0)  # Batch size 1

In [ ]:
# Obtain the BERT embeddings
with torch.no_grad():
    t_outputs1 = model(t_input_ids1)
    t_outputs2 = model(t_input_ids2)
    t_embeddings1 = t_outputs1.last_hidden_state[:, 0, :]  # [CLS] token
    t_embeddings2 = t_outputs2.last_hidden_state[:, 0, :]  # [CLS] token


In [ ]:
# Calculate similarity
t_similarity_score = cosine_similarity(t_embeddings1, t_embeddings2)
print("Similarity Score:", t_similarity_score)

##Реализация

In [24]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
model = AutoModel.from_pretrained("deepvk/USER-bge-m3")

In [26]:
# Example sentences (already preprocessed)
tokens_answer = []
for e in range(len(answers)):
  tokens_answer.append(answers[e].split())

tokens_message = []
for e in range(len(message)):
  tokens_message.append(message[e].split())

In [27]:
# Convert tokens to input IDs
input_ids_messages = []
input_ids_answers = []

for e in range(len(tokens_answer)):
  input_ids_messages.append(torch.tensor(tokenizer.convert_tokens_to_ids(tokens_message[e])).unsqueeze(0))  # Batch size 1
  input_ids_answers.append(torch.tensor(tokenizer.convert_tokens_to_ids(tokens_answer[e])).unsqueeze(0))  # Batch size 1

In [28]:
# Obtain the BERT embeddings
outputs_messages = []
outputs_answers = []
embeddings_messages = []
embeddings_answers = []
with torch.no_grad():
    for e in range(len(tokens_answer)):
      outputs_messages.append(model(input_ids_messages[e]))
      outputs_answers.append(model(input_ids_answers[e]))
      embeddings_messages.append(outputs_messages[e].last_hidden_state[:, 0, :])  # [CLS] token
      embeddings_answers.append(outputs_answers[e].last_hidden_state[:, 0, :])  # [CLS] token

In [29]:
# Calculate similarity
similarity_score = []
i = 0
average_similarity_score = 0
df_sim_score = pd.DataFrame(columns=['Answer', 'LLM_Answer', 'Similarity Score'])
for e in range(len(tokens_answer)):
  similarity_score.append(cosine_similarity(embeddings_messages[e], embeddings_answers[e]))
  average_similarity_score += similarity_score[e]
  i +=1
  new_row = {'Answer': answers[e], 'LLM_Answer': message[e], 'Similarity Score': similarity_score[e]}
  df_sim_score = pd.concat([df_sim_score, pd.DataFrame([new_row])], ignore_index=True)
print("Average Similarity Score = ", average_similarity_score / i)
df_sim_score

Average Similarity Score =  [[0.41952765]]


,Answer,LLM_Answer,Similarity Score
0,прямые ограничения по допуску иностранных граж...,В значимом объекте не допускается наличие удал...,[[0.1976471]]
1,первичное моделирование угроз безопасности инф...,Моделирование угроз должно выполняться до разр...,[[0.63362443]]
2,"да, допускается, однако в отношении используем...",Не знаю,[[0.42731145]]


#BERTScore

In [30]:
from torchmetrics.functional.text.bert import bert_score
preds = message
target = answers
df_BERTScore = pd.DataFrame(columns=['Answer', 'LLM_Answer', 'Precision', 'Recall', 'F1'])
b_score = bert_score(preds=preds , target=target, model_name_or_path="deepvk/USER-bge-m3")
for e in range(len(message)):
  new_row = {'Answer': answers[e], 'LLM_Answer': message[e], 'Precision': b_score['precision'][e], 'Recall': b_score['recall'][e], 'F1': b_score['f1'][e]}
  df_BERTScore = pd.concat([df_BERTScore, pd.DataFrame([new_row])], ignore_index=True)
df_BERTScore

,Answer,LLM_Answer,Precision,Recall,F1
0,прямые ограничения по допуску иностранных граж...,В значимом объекте не допускается наличие удал...,tensor(0.7262),tensor(0.6022),tensor(0.6584)
1,первичное моделирование угроз безопасности инф...,Моделирование угроз должно выполняться до разр...,tensor(0.7115),tensor(0.6017),tensor(0.6520)
2,"да, допускается, однако в отношении используем...",Не знаю,tensor(0.5724),tensor(0.4781),tensor(0.5210)


In [63]:
#На что опирался
from torchmetrics.functional.text.bert import bert_score
preds = ["hello there", "general kenobi", "я был здесь"]
target = ["hello there", "master kenobi", "я был тут"]
q = bert_score(preds, target)
q

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The argument `model_name_or_path` was not specified while it is required when default `transformers` model are used.It is, therefore, used the default recommended model - roberta-large.
  warnings.warn(*args, **kwargs)  # noqa: B028
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': tensor([1.0000, 0.9961, 0.9787]),
 'recall': tensor([1.0000, 0.9961, 0.9832]),
 'f1': tensor([1.0000, 0.9961, 0.9809])}